**Move all the files fom inside a folder to outside**

```
find . -name '*.fq.gz' -exec mv {} . \;
find . -name '*.txt' -exec mv {} . \;
find . -depth -type d -empty -exec rmdir {} \;
```

**Create the alignment.csv file**

```
(echo 'samplename,description'; for f in raw/*fq.gz; do readlink -f $f | perl -pe 's/(.*?_(S[0-9]+)_.*)/\1,\2/'; done) > alignment.csv
```

**Prepare the O2.yaml file**

```
cd ..
vim O2.yaml

details:
  - analysis: RNA-seq
    genome_build: hg38
    disambiguate: [mm10]
    algorithm:
      aligner: hisat2
      strandedness: unstranded

upload:
  dir: ../final
```

**Intiate bcBio**

```
module load bcbio/latest
unset PYTHONPATH
bcbio_nextgen.py -w template O2.yaml alignment.csv raw/
```

**Create Submission script to O2**

```
vim submit_bcbio.sh

#!/bin/sh
#SBATCH -p priority
#SBATCH -J gw
#SBATCH -o run.o
#SBATCH -e run.e
#SBATCH -t 10-00:00
#SBATCH --cpus-per-task=20
#SBATCH --mem=150G
#SBATCH --mail-type=END         # Type of email notification- BEGIN,END,FAIL,ALL
#SBATCH --mail-user=ajitj_nirmal@dfci.harvard.edu   # Email to which notifications will be sent

export PATH=/n/app/bcbio/tools/bin:$PATH
bcbio_nextgen.py ../config/alignment.yaml \
    -n 24 -t local
```

**Submit job to O2 for processing**

```
cp submit_bcbio.sh alignment/work
cd alignment/work
sbatch submit_bcbio.sh
```
